In [1]:
import math
import pandas as pd
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def showImages(images):
    count = 0
    for img in images:
        windowName = "img" + str(count)
        cv2.namedWindow(windowName, cv2.WINDOW_NORMAL)
        cv2.imshow(windowName,img)
        cv2.waitKey(0)
        count += 1
    cv2.destroyAllWindows()

In [32]:
def detectColonies(imagePath):
    #Binary Mask
    plateImage = cv2.imread(imagePath,0)
    image = cv2.fastNlMeansDenoising(plateImage,None,10,7,21)
    adapGauss = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 2)
    
    #Blob Detection
    params2 = cv2.SimpleBlobDetector_Params()
    params2.filterByColor = True
    params2.blobColor = 255
    params2.filterByInertia = True
    params2.minInertiaRatio = 0.01
    params2.filterByConvexity = True
    params2.minConvexity = 0.9
    detector = cv2.SimpleBlobDetector_create(params2)
    
    keypoints = detector.detect(adapGauss)
    binaryKeypoints = cv2.drawKeypoints(adapGauss, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    originalKeypoints = cv2.drawKeypoints(image, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    print("%d Colonies Detected" % (len(keypoints)))
    images = []
    images.append(binaryKeypoints)
    images.append(originalKeypoints)
    showImages(images)
    return (keypoints, adapGauss)

In [31]:
def dist(p1, p2):
    return math.sqrt((p2[0] -  p1[0])**2 + (p2[1] - p1[1])**2)

class Blob:
    def __init__(self, coord, whitePix, darkPix):
        self.coord = coord
        self.whitePix = whitePix
        self.darkPix = darkPix

def analyzeColonies(keyPoints, binaryImage):
    R = 32
    phi = 0.1
    width = len(binaryImage)
    height = len(binaryImage[0])
    blobList = []
    for keyPoint in keyPoints:
        # Obtaining the x,y pixel coordinates of the blob
        x = (int) (keyPoint.pt[0])
        y = (int) (keyPoint.pt[1])
        coord = [x, y]
        # Starting x and y coordinates for the segmented box
        lowX = x - R
        lowY = y - R

        # discarding blobs too close to window boundaries
        if lowX < 0 or lowY < 0 or lowX + 2*R > width or lowY + 2*R > height: 
            continue 
        r = (int) (keyPoint.size * (1+phi))/2
        # dark percentage inside = darkCount/insideCount
        # white percentage outside = whiteCOunt/outsideCount
        insideCount = 0
        outsideCount = 0
        whiteCount = 0
        darkCount = 0
        for i in range(lowX, lowX + 2*R):
            for j in range (lowY, lowY + 2*R):
                # Do not count pixels that are outside larger circle
                if dist([x,y], [i,j]) >= R:
                    continue
                # Check for dark pixels inside blob
                elif dist([x,y], [i,j]) <= r: 
                    insideCount += 1
                    if binaryImage[j, i] == 0:
                        darkCount += 1
                # Check for white pixels outside blob
                else:
                    outsideCount += 1
                    if binaryImage[j, i] == 255: 
                        whiteCount += 1
        whitePix = whiteCount*100/outsideCount
        darkPix = darkCount*100/insideCount
        blobList.append(Blob(coord, whitePix, darkPix))
    print("%d Viable Colonies Analyzed" % len(blobList))
    return blobList

In [35]:
def rankColonies(blobList):
    darkCutoff = np.percentile(np.array([blob.darkPix for blob in blobList]), 95)
    whiteCutoff = np.percentile(np.array([blob.whitePix for blob in blobList]), 50)
    for i,blob in enumerate(blobList):
        if blob.darkPix > darkCutoff or  blob.whitePix > whiteCutoff:
            blobList.remove(blob)
    blobList.sort(key=lambda x: x.whitePix)
    print("%d Viable Colonies Extracted" % len(blobList))
    return blobList

In [33]:
keypoints, binaryImage = detectColonies("Plate Images/image-00000.jpg")

712 Colonies Detected


In [34]:
blobList = analyzeColonies(keypoints, binaryImage)

679 Viable Colonies Analyzed


In [36]:
updatedList = rankColonies(blobList)

451 Viable Colonies Extracted
